In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
from pycaret.classification import *

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test_x.csv")
submission = pd.read_csv("sample_submission.csv")

In [3]:
train = train[train['familysize'] <=23]

In [4]:
QA = []
wf = []
wr = []
tp = []
QE = []
for i in train.columns: 
  if i.endswith("A"):
    QA.append(i)
  if i.startswith("wf"):
    wf.append(i)
  if i.startswith("wr"):
    wr.append(i)
  if i.startswith("tp"):
    tp.append(i)
  if i.endswith("E"):
    QE.append(i)

In [5]:
train['QA_trash'] = 0 

for i in range(0,len(train)):
  if len(train[QA].iloc[i].unique()) <= 2:
    train['QA_trash'].iloc[i] = 1

C:\Users\SukWoo\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
train = train[train['QA_trash'] == 0]

In [7]:
train[tp] = 7 - train[tp]
test[tp] = 7 - test[tp]

In [8]:
train['time'] = train[QE].sum(axis = 1)
test['time'] = test[QE].sum(axis = 1)

In [9]:
train['time_category'] = 0
train['time_category'] = train['time'].apply(lambda x : 1 if x < 18383 else 2 if x<23763 else 3 if x < 400000 else 4)
test['time_category'] = test['time'].apply(lambda x : 1 if x < 18383 else 2 if x<23763 else 3 if x < 400000 else 4)

In [10]:
train['wr_sum'] = train[wr].sum(axis=1)
train['wf_sum'] = train[wf].sum(axis=1)
test['wr_sum'] = test[wr].sum(axis=1)
test['wf_sum'] = test[wf].sum(axis=1)

In [11]:
train = train[['age_group', 'education', 'engnat', 'familysize', 'gender', 'married', 'race', 'religion','time_category', 'wr_sum', 'wf_sum','tp01', 'tp02', 'tp03', 'tp04',
       'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban', 'voted']]

test = test[['age_group', 'education', 'engnat', 'familysize', 'gender', 'married', 'race', 'religion','time_category', 'wr_sum', 'wf_sum','tp01', 'tp02', 'tp03', 'tp04',
       'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban']]

In [12]:
def tp_change_plus(data, tp_number, get_number):
  for i in range(0,len(data)):
    if data['tp{}'.format(tp_number)].iloc[i] == 0 and data['tp{}'.format(get_number)].iloc != 0:
      data['tp{}'.format(tp_number)].iloc[i] = 8 - data['tp{}'.format(get_number)].iloc[i]

In [13]:
tp_change_plus(train, '01', '06')
tp_change_plus(train, '02', '07')
tp_change_plus(train, '03', '08')
tp_change_plus(train, '04', '09')
tp_change_plus(train, '05', '10')

tp_change_plus(train, '06', '01')
tp_change_plus(train, '07', '02')
tp_change_plus(train, '08', '03')
tp_change_plus(train, '09', '04')
tp_change_plus(train, '10', '05')

tp_change_plus(test, '01', '06')
tp_change_plus(test, '02', '07')
tp_change_plus(test, '03', '08')
tp_change_plus(test, '04', '09')
tp_change_plus(test, '05', '10')

tp_change_plus(test, '06', '01')
tp_change_plus(test, '07', '02')
tp_change_plus(test, '08', '03')
tp_change_plus(test, '09', '04')
tp_change_plus(test, '10', '05')

In [14]:
train['wr_sum'] = (train['wr_sum'] - 6.5) / 6.5
test['wr_sum'] = (test['wr_sum'] - 6.5) / 6.5

train['wf_sum'] = (train['wf_sum'] - 1.5) / 1.5
test['wr_sum'] = (test['wf_sum'] - 1.5) / 1.5

train[tp] = (train[tp] - 3.5) / 3.5
test[tp] = (test[tp] - 3.5) / 3.5

In [15]:
clf = setup(data = train, target = 'voted')

Setup Succesfully Completed!


,Description,Value
0,session_id,5350
1,Target Type,Binary
2,Label Encoded,"1: 0, 2: 1"
3,Original Data,"(44518, 23)"
4,Missing Values,False
5,Numeric Features,13
6,Categorical Features,9
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [16]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Light Gradient Boosting Machine,0.6986,0.7685,0.649,0.7639,0.7017,0.4011,0.4067,0.4882
1,CatBoost Classifier,0.6953,0.7675,0.6586,0.7528,0.7025,0.3931,0.3968,22.65
2,Gradient Boosting Classifier,0.6967,0.7672,0.6458,0.7628,0.6993,0.3977,0.4034,7.5
3,Linear Discriminant Analysis,0.6913,0.7631,0.6588,0.7464,0.6998,0.3847,0.3879,0.3608
4,Logistic Regression,0.692,0.7624,0.6632,0.7451,0.7017,0.3856,0.3885,0.541
5,Ada Boost Classifier,0.6905,0.7605,0.6522,0.7489,0.6971,0.3837,0.3876,2.038
6,Extreme Gradient Boosting,0.6853,0.755,0.6657,0.7336,0.698,0.371,0.373,4.796
7,Extra Trees Classifier,0.6865,0.7549,0.6512,0.7431,0.6941,0.3754,0.3789,3.895
8,Naive Bayes,0.6789,0.7371,0.644,0.7354,0.6866,0.3604,0.3638,0.0479
9,Random Forest Classifier,0.6676,0.7295,0.6132,0.7345,0.6684,0.3406,0.3462,0.3359


In [17]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6995,0.7695,0.6499,0.7647,0.7027,0.403,0.4084
1,0.6976,0.7667,0.6373,0.7695,0.6972,0.4004,0.4076
2,0.6946,0.7724,0.6519,0.7555,0.6999,0.3925,0.3969
3,0.699,0.7649,0.6536,0.7615,0.7034,0.4014,0.4062
4,0.7046,0.7765,0.6576,0.7684,0.7087,0.4127,0.4179
Mean,0.6991,0.77,0.6501,0.7639,0.7024,0.402,0.4074
SD,0.0032,0.0041,0.0069,0.0051,0.0038,0.0065,0.0067


In [18]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.695,0.7653,0.641,0.7628,0.6967,0.3947,0.4007


In [19]:
final_model = finalize_model(blended)

In [20]:
predictions = predict_model(final_model, data = test)

In [21]:
submission['voted'] = predictions['Score']

In [22]:
submission.to_csv('submission_proba.csv', index = False)

In [24]:
submission['voted'].max()

0.9889

In [25]:
submission['voted'].min()

0.1035